In [22]:
import chipwhisperer as cw
import numpy as np
import scienceplots
import matplotlib.pyplot as plt

import simon_64_128_simulation
import correlations

from measurement import Measurements

# Import Trace

In [23]:
project = cw.import_project("./traces/14_simon_plain_leakage_50000/trace.zip", overwrite=True)

In [24]:
NUM_MEASUREMENTS = len(project.traces) // 2
VALS_PER_MEASUREMENT = len(project.traces[0].wave)

# Transform Trace Project into Measurement object

In [25]:
plaintexts1 = np.zeros((NUM_MEASUREMENTS, 2), dtype=np.uint32)
ciphertexts1 = np.zeros((NUM_MEASUREMENTS, 2), dtype=np.uint32)
powers1 = np.zeros((NUM_MEASUREMENTS, VALS_PER_MEASUREMENT), dtype=np.uint32)

plaintexts2 = np.zeros((NUM_MEASUREMENTS, 2), dtype=np.uint32)
ciphertexts2 = np.zeros((NUM_MEASUREMENTS, 2), dtype=np.uint32)
powers2 = np.zeros((NUM_MEASUREMENTS, VALS_PER_MEASUREMENT), dtype=np.uint32)

for i, trace in enumerate(project.traces):

    pt = trace.textin
    ct = trace.textout

    if i % 2 == 0:
        plaintexts1[i//2,0] = pt[0] << 24 | pt[1] << 16 | pt[2] << 8 | pt[3]
        plaintexts1[i//2,1] = pt[4] << 24 | pt[5] << 16 | pt[6] << 8 | pt[7]

        ciphertexts1[i//2,0] = ct[0] << 24 | ct[1] << 16 | ct[2] << 8 | ct[3]
        ciphertexts1[i//2,1] = ct[4] << 24 | ct[5] << 16 | ct[6] << 8 | ct[7]
        powers1[i//2,:] = trace.wave

    else:
        plaintexts2[i//2,0] = pt[0] << 24 | pt[1] << 16 | pt[2] << 8 | pt[3]
        plaintexts2[i//2,1] = pt[4] << 24 | pt[5] << 16 | pt[6] << 8 | pt[7]

        ciphertexts2[i//2,0] = ct[0] << 24 | ct[1] << 16 | ct[2] << 8 | ct[3]
        ciphertexts2[i//2,1] = ct[4] << 24 | ct[5] << 16 | ct[6] << 8 | ct[7]
        powers2[i//2,:] = trace.wave

measurements1 = Measurements(plaintexts1, ciphertexts1, powers1)
measurements2 = Measurements(plaintexts2, ciphertexts2, powers2)

In [26]:
mean1 = np.mean(measurements1.power, axis=0)
std1 = np.std(measurements1.power, axis=0)

mean2 = np.mean(measurements2.power, axis=0)
std2 = np.std(measurements2.power, axis=0)  


t = (mean1 - mean2) / np.sqrt((std1**2 + std2**2) / NUM_MEASUREMENTS)
print(t.shape)

print(np.max(t))
print(np.min(t))

(5000,)
625.2890994809942
-498.8969105287077


In [28]:

with plt.style.context(['science', 'ieee']):
    plt.plot(t, color="black", linewidth=0.5, linestyle="-")
    plt.yscale('linear')
    plt.axhspan(-4.5, 4.5, color='lightgray')
    plt.savefig("diagrams/leakage_over_time_plain.pdf")
    plt.savefig("diagrams/leakage_over_time_plain.pgf")